# Markov Decision Processes
Covers formalism, etc... These differ from bandits in two key ways: 

* There is modeled dependence on "context" - for bandits, we want to estimate the value $q_*(a)$, but in an MDP we also want to estimate the value as a function of state or context: $q_*(s, a)$.

* There is dependence on sequential decisions because our actions can change subsequent state...

Generally, an agent (learner or decision maker) interacts with an environment.  At each time step $t$ (can be continuous), the agent selects an action $a \in \mathcal{A}(s)$ (note dependence of action set on state).  The environment evolves in response to the action (i.e., subsequent state can depend on the action taken) => $S_{t+1}$, and also provides some scalar reward $R_{t+1} \in \mathbb{R}$ in response to the starting action and state, $A_t$ and $S_t \in \mathcal{S}$.  

An interesting practical question is - where do we draw the boundary between agent and environment?  "The general rule we follow is that anything that cannot be changed arbitrarily by the agent is considered to be outside of it and thus part of the environment.  We do not assumed that everything in the environment is unknown to the agent."  This is an important principle in formulating MDPs - the boundary is one of control, not knowledge!  For instance, in a robot control system, we probably want to draw the boundary at the signals sent to actuators since that is all the agent can arbitrarily control...  This is so even if the robot has pretty precise and accurate sensors telling us how many steps were taken by a step motor, for instance.  "the agent-environment boundary represents the limit of the agent's absolute control, not of its knowledge". 

This chapter introduces lots of notation so there's not much to be done for it except to slog through...


## The reward hypothesis
They make a big deal of this, and Michael Littman of Brown has a thoughtful video about this that is worth checking out...  

>That all of what we mean by goals and purposes can be well thought of as the maximization of the expected value of the cumulative sum of a received scalar signal (called reward).

This is a big assumption and a _sine qua none_ of RL, and we can often try to make this approximately true by expanding our notion of state...  Note that the reward signal is not where we want to impart prior knowledge to the agent about _how_ goals should be achieved!  

## MDP Dynamics 
Dynamics in MDPs refers to how the environment evolves over time in response to actions taken by the agent.  This evolution is stochastic, and is assumed to be governed _completely_ by a probability distribution: 

$$
p(s', r | s, a) = Pr{S_t = s', R_t = r | S_{t-1} = s, A_{t-1} = a} \text{  } \forall s', s \in \mathcal{S}, a \in \mathcal{A}(s)
$$

i.e., the dynamics specify a distribution over rewards and states conditional on starting states and actions.  Thus, 
$$
\sum_{s' \in \mathcal{S}} \sum_{r \in \mathbb{R}} p(s', r | s, a) = 1 \text{    } \forall s \in \mathcal{S}, a \in \mathcal{A}(s)
$$

This dependence only on the current state is why we refer to this as a Markov process.  "This is best viewed as a restriction not on the decision process, but on the _state_.  The state must include information about all aspects of the past agent-environment interaction that make a difference for the future".  In practice, we can usually make this at least approximately true by simply expanding our definition of state...

## Things defined in terms of the dynamics

### State transition probabilities
We are probably used to thinking of Markov processes as governed by a state transition function.  Given the dynamics function, we can do the usual manipulations of probability distributions to arrive at various interesting things, like the implied state transition probabilities.  This is a probability distribution over subsequent states given a starting state and action, and we can get this by marginalizing the dynamics over rewards: 

$$
p(s' | s, a) = Pr{S_t = s' | S_{t-1} = s, A_{t-1} = a} = \sum_{r \in \mathbb{R}} p(s', a | s, a)
$$

### Expected rewards for state-action pairs
It would also be interesting to know what the expected reward is for taking action $a$ in state $s$...  If we have the dynamics function, we can calculate this: 

$$
r(s, a): \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R} = \
\mathbb{E}[R_t | S_{t-1} = s, A_{t-1} = a] = \sum_{r \in \mathbb{R}} \sum_{s' \in \mathcal{S}} r p(s', r|s, a)
$$

We can also ask more granular questions like, what is the expected reward if we start in $s$, take action $a$, and end up in state $s'$?  

$$
\begin{align}
r(s, a, s'): \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow \mathbb{R} & = \
\mathbb{E}[R_t | S_{t-1} = s, A_{t-1} = a, S_t = s'] \\
& = \sum_{r \in \mathbb{R}}  r \frac{p(s', r|s, a)}{p(s'|s,a)}
\end{align}
$$

This expectation is defined in terms of the dynamics model and the state transition model. 

## Returns and episodes
### Rewards and episodic taks
How do we define the reward the agent is trying to maximize?  Say our task is something like, playing games of chess.  Each game has a natural end state, after which we probably want to reset the state to an initial state (or more generally, a draw from a distribution of initial states).  When the task is naturally broke up into subsequences, we call the subsequences _episodes_, and we formulate the problem to have a special _terminal state_ defining the ends of episodes.  These number of steps it takes to reach the terminal state is a random variable, $T$.  In this setting, it is also natural to define the _return_ at time $t$ as the sum of rewards received in times steps from $t$ up to and including $T$ (note - this elides a key question of what actions we take in those time steps!): 

$$
G_t = R_{t+1} + R_{t+2} + \dots + R_{T}
$$

### Continuous tasks
What do we do if our task is continuous, i.e., there isn't a good way to break it up into subsequences?  In other words, $T \rightarrow \infty$?  This immediately presents a problem - the return as we defined it is unbounded, and there is no good basis for saying that following one "policy" versus another is better or worse (we could introduce an arbitrary cut off point where we decide but that defeats ignores the possibility that we could do better with a bit more delayed gratification).  In this case, we use a _discount factor_, $0 \le \gamma < 1$ and define the discounted return: 

$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots = \sum_{k=0}^{\infty} \gamma^k R_{t + k + 1}
$$

As long as $R_t$ is bounded, this is finite, and has upper bound $\frac{\sup{\{R\}}}{1 - \gamma}$ since $\sum_{k=0}^{\infty} \gamma^k = \frac{1}{1-\gamma}$.  Practically, bigger $\gamma$ encourages longer term considerations; $\gamma = 0$ is very short sighted and we are treating this like a bandit problem.  On the other hand, $\gamma = 1$ gets us back to the definition of return we used for episodic tasks.

### Unified notation
In practice, we can use the notation for continuous tasks for episodic tasks by adopting a convention of a terminal absorbing state that transitions only to itself and has a reward of zero.  Then the math works out fine.  This saves us from introducing another index over episodes, which would be a pain in the ass.